# SingleStore and Llama 3 RAG Quickstart

This notebook provides an example of how to use SingleStore as a vector database in conjunction with Llama 3. We'll be using Replicate to host an instance of Llama 3 ov

In [133]:
!pip install -q -U langchain singlestoredb langchain-openai --quiet

In [2]:
!pip install replicate --quiet


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install langchain_community --quiet

In [125]:
from getpass import getpass

import os

REPLICATE_API_TOKEN = getpass()

os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

 ········


In [138]:
OPENAI_API_KEY = getpass()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

 ········


In [126]:
from langchain.chains import LLMChain
from langchain_community.llms import Replicate
from langchain_core.prompts import PromptTemplate

## Initialize Llama 3 in Replicate

In [131]:
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: What is SingleStoreDB?
Assistant:
"""
llm(prompt)

'SingleStoreDB is a distributed relational database that is designed to handle large amounts of data and scale horizontally. It is optimized for real-time analytics and machine learning workloads.\n\nSingleStoreDB is built from the ground up to be a cloud-native database. It is designed to take advantage of cloud computing, including scalability, high availability, and cost-effectiveness.\n\nSome of the key features of SingleStoreDB include:\n\n* Scalability: SingleStoreDB can scale horizontally to handle large amounts of data and increasing workloads.\n* High availability: SingleStoreDB is designed to provide high availability, ensuring that your database is always available and accessible.\n*'

## RAG over audio files using SingleStoreDB

Now we will embed the text descriptions of the audio file(s). This allows us to search and retrieve relevant files for RAG later.

In [164]:
from langchain.vectorstores import SingleStoreDB
import os

from langchain_openai import OpenAIEmbeddings

os.environ["SINGLESTOREDB_URL"] = f'{connection_user}:{connection_password}@{connection_host}:{connection_port}/{connection_default_database}'

In [223]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://aws.amazon.com/blogs/aws/metas-llama-3-models-are-now-available-in-amazon-bedrock/")
data = loader.load()

In [224]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
all_splits = text_splitter.split_documents(data)

In [225]:
vectorstore=SingleStoreDB.from_documents(documents=all_splits, table_name="test6", embedding=OpenAIEmbeddings())

In [227]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": "How do you use Llama 3 models in the AWS console?"})

{'query': 'How do you use Llama 3 models in the AWS console?',
 'result': 'To use Llama 3 models in the AWS console, you can follow these steps:\n\n1. Log in to the AWS Management Console.\n2. Navigate to the Amazon SageMaker console.\n3. Click on the "Models" tab.\n4. Click on the Llama 3 model you want to use.\n5. In the "Model details" page, scroll down to the "Use in a SageMaker notebook" section.\n6. Click on the "Use" button.\n7. Review the following steps from the previous response.\n\nYou can also use the Llama 3 model to generate a new Llama 3 model to'}